<h1> Sequential Graph: Building a Pipeline </h1>

### Intro
Real-world tasks are rarely single-step. Usually, you need a pipeline:
1. Do Step A.
2. Pass result to Step B.
3. Do Step B.

**Goal:** Build a Sequential Graph where **Node A** prepares a message and **Node B** finishes it.

### Step 1: Imports
Standard boilerplate.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph

### Step 2: The State
We need a state that holds our inputs (`name`, `age`) and the variable we are building up (`result`).

In [ ]:
class AgentState(TypedDict):
    name: str   # Input 1
    age: str    # Input 2
    result: str # The accumulator string

### Step 3: Define The Nodes
We will have two workers in our factory line.

**1. `firstNode`**: 
- Takes the `name`.
- Starts the `result` string.

**2. `secondNode`**:
- Reads the `result` created by `firstNode`.
- Appends the `age` information to it.

In [ ]:
def firstNode(state: AgentState) -> AgentState:
    """
    Step 1: Adds the name to the result
    """ 
    print("--- Executing First Node ---")
    state["result"] = f"Hey {state["name"]}, "
    return state

def secondNode(state: AgentState) -> AgentState:
    """ 
    Step 2: Appends the age to the result.
    """ 
    print("--- Executing Second Node ---")
    # Notice we are using state['result'] which was set by the previous node
    # This is how we pass information down the pipeline!
    state["result"] = state["result"] + f"Your age is: {state["age"]} !"
    return state

### Step 4: Connecting the Dots
This is the crucial part. 
Instead of `set_finish_point`, we use `add_edge`.

`graph.add_edge("first", "second")` literally means: **"When 'first' is done, go immediately to 'second'"**.

In [ ]:
graph = StateGraph(AgentState)

# 1. Add nodes
graph.add_node("first", firstNode)
graph.add_node("second", secondNode)

# 2. Set Entry
graph.set_entry_point("first")

# 3. Connect them!
graph.add_edge("first", "second") 

# 4. Set Finish
graph.set_finish_point("second")

# 5. Compile
app = graph.compile()

### Step 5: Verify the Graph
Look at the arrow. It should point from `first` -> `second`.

In [ ]:
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))

### Step 6: Run the Pipeline
We provide all necessary input data. The graph handles the ordering.

In [ ]:
input_data = {
    "name": "Debu", 
    "age": 22
}

result = app.invoke(input_data)

### The Result
The final string is a combination of work done by both nodes.

In [ ]:
print(result["result"])

### Explanation
1. **Start**: Graph went to `firstNode`.
2. **First Node**: Created "Hey Debu, ".
3. **Edge**: Graph followed the edge to `secondNode`.
4. **Second Node**: Took "Hey Debu, " and added "Your age is: 22 !".
5. **Finish**: Graph stopped.